**Случайный лес**

*Случайный лес* — алгоритм машинного обучения, представляющий собой ансамбль независимых решающих деревьев.

Алгоритм построения случайного леса, состоящего из N деревьев, выглядит следующим образом:

Для каждого n = 1, … , N:
1. Сгенерировать выборку $X_n$ с помощью бутстрэпа;
2. Построить решающее дерево bn по выборке $X_n$:
- по заданному критерию мы выбираем лучшии признак, делаем разбиение в дереве по нему и так до исчерпания выборки
- дерево строится, пока в каждом листе не более $n_{min}$ объектов или пока не достигнем определенной высоты дерева
- при каждом разбиении сначала выбирается m случайных признаков из n исходных,
и оптимальное разделение выборки ищется только среди них.







Для задачи кассификации финальное решение выбирается голосованием по большинству, а в задаче регрессии - средним.

**Постановка задачи** 

Для набора данных по раку груди — Scikit-learn «breast cancer» построить классификационную модель прогнозирования рака, используя алгоритм "Случайный лес". Оценить точность модели.

**Описание структуры исходных данных**

В алгоритме постороения случайного дерева используются решающие деревья. Дерево решений — метод представления решающих правил в определенной иерархии, включающей в себя элементы двух типов — узлов (node) и листьев (leaf). Узлы включают в себя решающие правила и производят проверку примеров на соответствие выбранного атрибута обучающего множества.

Примеры попадают в узел, проходят проверку и разбиваются на два подмножества:

1. те, которые удовлетворяют установленному правилу
2. те, которые не удовлетворяют установленному правилу

Далее к каждому подмножеству снова применяется правило, процедура повторяется, пока не будет достигнуто условие остановки алгоритма.

**Описание используемых функций и их параметров**

pandas.DataFrame(data=None, columns=None) - создает дата фрейм

>- data - двумерный массив с данными.
>- columns - название столбцов.

DataFrame.head(n) - функция, возврашающая первые n строк дата фрейма.

>- n: int число строк(по умолчанию 5).

pandas.DataFrame.drop(labels = None, axis = 0) 

>- labels - метки строки или столбца, которые необходимо удалить.
>- axis - удалять метки из строк(0) или столбцов(1).

sklearn.model_selection.train_test_split(arrays1, arrays2, test_size=None, random_state=None, stratify=None)

>- arrays1 - образец набора функций для разделения.
>- arrays2 - образец результата, который нужно разделить.
>- test_size - доля выборок, если это целое число, это количество выборок.
>- random_state - это начальное число случайного числа.
>- stratify -  разбивает данные послойно, используя это как метки класса.

StandardScaler() - создаёт объект функции StandardScaler()

object.fit_transform(data) - преобразует данные таким образом, что его распределение будет иметь среднее значение 0 и стандартное отклонение 1

>- data - двумерный массив с данными.

object.fit_transform(data) - Стандартизирует данные

>- data - двумерный массив с данными.

RandomForestClassifier() - Классификатор случайного леса.

RandomForestClassifier().fit(X, y) - строит лес деревьев из тренировочного набора (X, y).

>- X - обучающие входные образцы.
>- y - целевые значения.

RandomForestClassifier().predict(X_test_scaled) - прогнозирует класс входной выборки.

>- X - входные образцы.

**Решение задачи**

**1. Импорт библиотек**

In [2]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

**2. Импорт набора данных**

1. data - необходимые данные для предсказания.
2. target - это целевые данные, которые мы хотим предсказать.

Последний столбец "cancer"(рак) - это целевая переменная, значение которой мы хотим предсказать, используя модель. Если заболевания нет - пишем 0, иначе 1.

In [3]:
# Выбираем признаки
columns = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']
# Загружаем набор данных
dataset = load_breast_cancer()
# Считываем DataFrame
data = pd.DataFrame(dataset['data'], columns=columns)
data['cancer'] = dataset['target']

# Для представления набора данных, можно вывести первые 10 строчек
display(data.head(10))


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,cancer
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,0
6,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,0
7,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,0
8,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,0
9,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,0


**3. Подготовка данных для обучения**

В этом разделе будут выполнены две задачи. Первая задача состоит в том, чтобы разделить данные на наборы “атрибутов” и “меток”. Полученные данные затем делятся на обучающие и тестовые наборы. 

Мы разделим набор на две равные части — 50% — учебные данные и 50% — тестовые.   Для обеспечения того, чтобы и в учебном, и в проверочном наборах данных присутствовало бы то же соотношение 0 и 1, что и в исходном наборе данных, устанавливаем stratify=y.

In [4]:
# Атрибуты
X = data.drop('cancer', axis=1) 
# Метки 
y = data['cancer'] 
# Делим на обучающие и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state = 2020, stratify=y)

**4. Масшабирование данных**

Перед тем, как приступать к моделированию, нужно масштабировать данные. 
У нас различные величины выражены в разных единицах измерения, поэтому надо выполнить масштабирование. Эта процедура позволяет организовать «честную схватку» между признаками при определении их важности. Кроме того, мы конвертируем y_train из типа данных Pandas Series в массив NumPy для того чтобы позже модель смогла бы работать с соответствующими целевыми показателями.

In [5]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
y_train = np.array(y_train)

**5. Обучение модели**

После масштабирования будем обучать модель на учебном наборе. Для этого сначала инициализируем модель. После этого обучим её на масштабированных данных. 

Далее оценим точность построенной модели. Создадим матрицы ошибок для модели и узнаем о том, как хорошо она смогла определить рак груди.

In [6]:
# Инициализация модели
rfc = RandomForestClassifier()
# Обучение модели
rfc.fit(X_train_scaled, y_train)



RandomForestClassifier()

**Визуализация результата**

In [7]:
# Рассчет точности
y_pred = rfc.predict(X_test_scaled)
# Построение матрицы
conf_matrix_baseline = pd.DataFrame(confusion_matrix(y_test, y_pred), index = ['Фактически 0', 'Фактически 1'], columns = ['Предсказано 0', 'Предсказано 1'])
# Вывод на экран
display(conf_matrix_baseline)
display('Точность модели, построенной с помощью алгоритма \"Случайный лес\"', recall_score(y_test, y_pred))

,Предсказано 0,Предсказано 1
Фактически 0,96,10
Фактически 1,10,169


'Точность модели, построенной с помощью алгоритма "Случайный лес"'

0.9441340782122905

**Интерпретация результата**

Точность модели составляет более 90%, что очень хорошо, учитывая, что мы имеем дело с диагнозированием рака!

**Задания для самостоятельной работы**

1. Для набора данных о ценах на жилье в Бостоне(from sklearn.datasets import load_boston) построить классификационную модель прогнозирования цен на жилье на основе атрибутов: количество комнат, уровень преступности в городе.  Оценить точность модели. Для создания модели использовать алгоритм "Случайный лес".

2. Для набора данных о о цветках ириса(from sklearn.datasets import load_iris) построить классификационную модель определяющую тип цветка на основе атрибутов: длинa и ширина чашелистика, длина и ширина лепестка. Оценить точность модели. Для создания модели использовать алгоритм "Случайный лес".